In [ ]:
#flag{Welcome_to_IOAI}, xor key=ioai
#lfkmq]ofiegoU~eUCEKCw <- encoded hash to be given

In [ ]:
!pip install torch==2.3.0+cu121

In [1]:
import torch
import torch.nn as nn

In [2]:
torch.manual_seed(42)

In [3]:
i_data = torch.zeros((26,), dtype=torch.float32)
i_data[8] = 100 #correct alphabet index will be as large as possible, the rest 0

In [4]:
#ANN - i

class SimpleANN(nn.Module):
    def __init__(self, input_size=26, hidden_size=64, output_size=50):
        super(SimpleANN, self).__init__()
        
        # Define layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input to hidden layer
        self.relu = nn.ReLU()                         # Activation function
        self.fc2 = nn.Linear(hidden_size, output_size) # Hidden layer to output

        # Initialize weights with a specific random seed
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights with random values
        nn.init.normal_(self.fc1.weight, mean=0.0, std=0.02)  # Normal distribution for fc1
        nn.init.normal_(self.fc2.weight, mean=0.0, std=0.02)  # Normal distribution for fc2

        # Initialize biases to zero (optional)
        nn.init.constant_(self.fc1.bias, 0.0)
        nn.init.constant_(self.fc2.bias, 0.0)

    def forward(self, x):
        # Forward pass
        x = self.fc1(x)  # Input layer to hidden layer
        x = self.relu(x) # Activation function
        x = self.fc2(x)  # Hidden layer to output layer
        return x

model = SimpleANN()
i_output = model(i_data)

In [12]:
o_data = torch.zeros((1, 1, 26), dtype=torch.float32)
o_data[0][0][14] = 100 #correct alphabet index will be as large as possible, the rest 0

In [13]:
#RNN - o
class SimpleRNN(nn.Module):
    def __init__(self, input_size=26, hidden_size=64, output_size=50, num_layers=1):
        super(SimpleRNN, self).__init__()
        
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer for the output
        self.fc = nn.Linear(hidden_size, output_size)

        # Initialize weights with a specific random seed
        self._initialize_weights()

    def _initialize_weights(self):
        # Custom weight initialization for reproducibility
        torch.manual_seed(42)

        # Initialize RNN weights
        for name, param in self.rnn.named_parameters():
            if 'weight' in name:
                nn.init.kaiming_normal_(param, mode='fan_out', nonlinearity='relu')
            elif 'bias' in name:
                nn.init.constant_(param, 0.0)

        # Initialize fully connected layer weights
        nn.init.xavier_uniform_(self.fc.weight)  # Xavier initialization for fc layer
        nn.init.constant_(self.fc.bias, 0.0)     # Set biases to zero

    def forward(self, x, hidden):
        # Forward pass through RNN
        out, hidden = self.rnn(x, hidden)  # RNN layer
        out = self.fc(out[:, -1, :])       # Fully connected layer (take last time step's output)
        return out, hidden

hidden_size = 64  # Hidden state size
batch_size = 1        # Batch size
num_layers = 1        # Number of RNN layers

model = SimpleRNN()
hidden = torch.zeros(num_layers, batch_size, hidden_size)
o_output, _ = model(o_data, hidden)

In [14]:
a_data = torch.zeros((26,), dtype=torch.float32)
a_data[0] = 100 #correct alphabet index will be as large as possible, the rest 0

In [15]:
#MLP - a

class SimpleMLP(nn.Module):
    def __init__(self, input_size=26, hidden_size=64, output_size=50):
        super(SimpleMLP, self).__init__()
        
        # Define layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input to hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) # Hidden layer to hidden layer
        self.fc3 = nn.Linear(hidden_size, output_size)  # Hidden layer to output layer
        self.relu = nn.ReLU()  # Activation function
        
        # Initialize weights with a specific random seed
        self._initialize_weights()

    def _initialize_weights(self):
        # Custom weight initialization for reproducibility
        torch.manual_seed(42)  # Optional: redundant if already set globally

        # Initialize fully connected layer weights with Xavier (Glorot) initialization
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)
        
        # Initialize biases to zero for fc layers
        nn.init.constant_(self.fc1.bias, 0.0)
        nn.init.constant_(self.fc2.bias, 0.0)
        nn.init.constant_(self.fc3.bias, 0.0)

    def forward(self, x):
        # Forward pass
        x = self.fc1(x)        # Input layer to first hidden layer
        x = self.relu(x)       # ReLU activation
        x = self.fc2(x)        # First hidden layer to second hidden layer
        x = self.relu(x)       # ReLU activation
        x = self.fc3(x)        # Second hidden layer to output layer
        return x

model = SimpleMLP()
a_output = model(a_data)

In [16]:
i2_data = torch.zeros((26,), dtype=torch.float32)
i2_data[8] = 100 #correct alphabet index will be as large as possible, the rest 0

In [17]:
#KAN - i
class SimpleKAN(nn.Module):
    def __init__(self, input_size=26, hidden_size=64, output_size=50):
        super(SimpleKAN, self).__init__()
        
        # Define layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input to hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) # Hidden layer to hidden layer
        self.fc3 = nn.Linear(hidden_size, output_size)  # Hidden layer to output layer
        self.relu = nn.ReLU()  # Activation function

        # Attention mechanism parameters
        self.attention = nn.Linear(hidden_size, hidden_size)  # Attention weights

        # Initialize weights with a specific random seed
        self._initialize_weights()

    def _initialize_weights(self):
        # Custom weight initialization for reproducibility
        torch.manual_seed(42)  # Optional: redundant if already set globally

        # Initialize fully connected layer weights with Xavier (Glorot) initialization
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)
        
        # Initialize attention weights with Xavier initialization
        nn.init.xavier_uniform_(self.attention.weight)
        
        # Initialize biases to zero for fc layers
        nn.init.constant_(self.fc1.bias, 0.0)
        nn.init.constant_(self.fc2.bias, 0.0)
        nn.init.constant_(self.fc3.bias, 0.0)
        nn.init.constant_(self.attention.bias, 0.0)

    def forward(self, x):
        # Forward pass
        x = self.fc1(x)            # Input layer to first hidden layer
        x = self.relu(x)           # ReLU activation
        x = self.fc2(x)            # First hidden layer to second hidden layer
        x = self.relu(x)           # ReLU activation

        # Apply attention mechanism
        attention_weights = self.attention(x)  # Attention mechanism
        x = x * attention_weights  # Apply attention weights

        x = self.fc3(x)            # Second hidden layer to output layer
        return x

model = SimpleKAN()
i2_output = model(i_data)

In [18]:
torch.save([i_output, o_output, a_output, i2_output], "outputs.pkl")